In [1]:
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from trade.search_trades import search_bull_put_spreads

# Add the project root directory to the Python path
project_root = os.path.abspath('..')
sys.path.insert(0, project_root)

In [2]:
# Now you can import the constants module
from config import constants_short_put_credit_spreads as configs
from config import constants_short_put_credit_spreads as configs

In [3]:
tickers = configs.TICKERS
margin = configs.MARGIN

low_days_cutoff = 1
high_days_cutoff = 10

In [4]:
final = search_bull_put_spreads(tickers, low_days_cutoff, high_days_cutoff, margin, )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********************

In [5]:
final.to_csv('../data/put_spread_trades.csv', index=False)

In [6]:
final.loc[final.ticker=="MSTR",['ticker','last_price','short_put_strike','long_put_strike','P(gain)', 'P(loss)','days','credit', 'market_credit',
       'risk', 'market_risk', 'c/r',"expected_annual_market_return"]].sort_values(by = "expected_annual_market_return").tail(10)

,ticker,last_price,short_put_strike,long_put_strike,P(gain),P(loss),days,credit,market_credit,risk,market_risk,c/r,expected_annual_market_return
840,MSTR,388.924988,385.0,377.5,0.737697,0.262303,2,3.975,3.55,3.525,3.95,1.127660,35.930029
876,MSTR,388.924988,382.5,377.5,0.763816,0.236184,2,2.525,2.10,2.475,2.90,1.020202,38.745833
949,MSTR,388.924988,377.5,365.0,0.858895,0.141105,2,4.550,4.25,7.950,8.25,0.572327,40.200609
877,MSTR,388.924988,382.5,375.0,0.784998,0.215002,2,3.775,3.20,3.725,4.30,1.013423,41.318459
948,MSTR,388.924988,377.5,367.5,0.851409,0.148591,2,3.775,3.45,6.225,6.55,0.606426,41.900613
839,MSTR,388.924988,385.0,380.0,0.715560,0.284440,2,3.050,2.45,1.950,2.55,1.564103,43.068265
947,MSTR,388.924988,377.5,370.0,0.842332,0.157668,2,3.025,2.70,4.475,4.80,0.675978,47.160003
945,MSTR,388.924988,377.5,375.0,0.823061,0.176939,2,1.250,0.90,1.250,1.60,1.000000,47.160003
946,MSTR,388.924988,377.5,372.5,0.834369,0.165631,2,2.100,1.80,2.900,3.20,0.724138,47.160003
801,MSTR,388.924988,387.5,380.0,0.704252,0.295748,2,5.050,4.30,2.450,3.20,2.061224,48.976574


In [7]:
final.columns

Index(['short_put_contract', 'short_put_strike', 'short_bid', 'short_ask',
       'short_price', 'long_put_contract', 'long_put_strike', 'long_bid',
       'long_ask', 'long_price', 'last_price', 'credit', 'market_credit',
       'risk', 'market_risk', 'c/r', 'market_c/r', 'ITM_return',
       'expected_return', 'P(success)', 'P(gain)', 'P(loss)',
       'break_even_return', 'expected_value', 'expected_annual_return',
       'expected_market_value', 'expected_market_return',
       'expected_annual_market_return', 'days', 'sma_10', 'sma_50', 'ticker'],
      dtype='object')

In [8]:
final['ticker'].value_counts().sort_values(ascending=False)

ticker
MSTR    886
MU      799
TDG     711
TSLA    696
FDX     628
       ... 
LUMN      1
XOM       1
PG        1
INTC      1
SLG       1
Name: count, Length: 223, dtype: int64